# Lab-05 YOLOv7 Inference

Please run the code with "VScode-devcontainer".

> You can find the tutorial provided by Visual Studio Code here :   
> [https://code.visualstudio.com/docs/devcontainers/containers](https://code.visualstudio.com/docs/devcontainers/containers)

## Import Required Libraries

In [10]:
from numpy import random
from pathlib import Path

import cv2
import torch
import matplotlib
import matplotlib.pyplot as plt

from models.experimental import attempt_load
from utils.datasets import LoadImages
from utils.general import non_max_suppression, scale_coords, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import time_synchronized

## Get Device

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Define Detect Function

In [12]:
def detect(weights, source, conf, iou_thres):
    # Directories
    save_dir = Path(increment_path(Path("runs/detect") / "exp", exist_ok=False))  # increment run
    save_dir.mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model

    # half precision only supported on CUDA
    if device.type != "cpu":
        model.half()  # to FP16

    # Set Dataloader
    dataset = LoadImages(source, img_size=640, stride=32)

    # Get names and colors
    names = model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != "cpu":
        model(torch.zeros(1, 3, 640, 640).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = 640
    old_img_b = 1

    for path, img, im0s, _ in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if device.type != "cpu" else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != "cpu" and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img)[0]

        # Inference
        t1 = time_synchronized()
        with torch.no_grad():  # Calculating gradients would cause a GPU memory leak
            pred = model(img)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, conf, iou_thres)
        t3 = time_synchronized()

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            s, im0 = "", im0s
            p = Path(path)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg

            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    label = f"{names[int(cls)]} {conf:.2f}"
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)

            # Print time (inference + NMS)
            print(f"{s}\nDone. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS")

            # Save results (image with detections)
            if dataset.mode == "image":
                cv2.imwrite(save_path, im0)
                print(f"The image with the result is saved in: {save_path}")

            # Change backend
            matplotlib.use("TkAgg")
            
            # Show Image
            plt.figure()
            plt.axis("off")
            plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
            plt.show()

## Detect

In [13]:
with torch.no_grad():
    detect(weights="yolov7.pt", source="inference/images/test2.jpg", conf=0.5, iou_thres=0.45)
    # detect(weights="runs/train/yolov7-custom13/weights/weight.pt", source="inference/images/fish1.jpg", conf=0.5, iou_thres=0.45)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
3 persons, 8 sheeps, 1 cell phone, 
Done. (6.6ms) Inference, (0.8ms) NMS
The image with the result is saved in: runs/detect/exp33/test2.jpg
